<div class="alert alert-warning">
    
<b>Disclaimer:</b> 
    
The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function  
- setting and running the fit  
- extracting and displaying information about the results  

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, note that the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.

</div>


# Lorentzian + Isotropic Rotational diffusion &lowast; Resolution with bumps

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to use a combination of models from the QENS library <em>i.e.</em> <b>Lorentzian</b>  and <b>IsotropicRotationalDiffusion</b> models.
</div>

The data are a set of water data measured at IN5 (ILL).

**Reference:** J. Qvist, H. Schober and B. Halle, *J. Chem. Phys.* **134**, 144508 (2011)

### Physical units
For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.


## Import libraries

In [ ]:
import ipywidgets
import h5py
import QENSmodels
import numpy as np
from scipy.integrate import simps
import bumps.names as bmp
from bumps import fitters
from bumps.formatnum import format_uncertainty_pm
import matplotlib.pyplot as plt

%matplotlib widget

## Setting of fitting

### Load reference data

In [ ]:
path_to_data = './data/'

# Data
# Wavelength 5 Angstrom
with h5py.File(path_to_data + 'H2O_293K_5A.hdf', 'r') as f:
    hw_5A = f['entry1']['data1']['X'][:]
    q_5A = f['entry1']['data1']['Y'][:]
    unit_w5A = f['entry1']['data1']['X'].attrs['long_name']
    unit_q5A = f['entry1']['data1']['Y'].attrs['long_name']
    sqw_5A = np.transpose(f['entry1']['data1']['DATA'][:])
    err_5A = np.transpose(f['entry1']['data1']['errors'][:])

# Resolution
# Wavelength 5 Angstrom
with h5py.File(path_to_data + 'V_273K_5A.hdf', 'r') as f:
    res_5A = np.transpose(f['entry1']['data1']['DATA'][:])

# Force resolution function to have unit area
# Wavelength 5 Angstrom
for i in range(len(q_5A)):
    area = simps(res_5A[:, i], hw_5A)
    res_5A[:, i] /= area

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True)

for i in range(len(q_5A)):
    ax[0].semilogy(hw_5A, sqw_5A[:,i], label=f"q={q_5A[i]:.1f}")
    ax[1].semilogy(hw_5A, res_5A[:,i], label=f"q={q_5A[i]:.1f}")

ax[0].set_title(r'Signal 5 $\AA$')
ax[0].grid()

ax[1].set_title(r'Resolution 5 $\AA$')
ax[1].set_xlabel(f"$\hbar \omega$")
ax[1].grid()

### Display units of input data 
Just for information in order to determine if a conversion of units is required before using the QENSmodels

In [ ]:
print(f"At 5 Angstroms, the names and units of `w` (`x`axis) and `q` are: {unit_w5A[0].decode()} and {unit_q5A[0].decode()}, respectively.")

### Create fitting model

In [ ]:
# Fit range -1 to +1 meV
idx_5A = np.where(np.logical_and(hw_5A > -1.0, hw_5A < 1.0))

# Fitting model
def model_convol(x, q, scale=1, center=0, hwhm=1, radius=1, DR=1, resolution=None):
    model = QENSmodels.lorentzian(
        x, 
        scale, 
        center, 
        hwhm
    ) + QENSmodels.sqwIsotropicRotationalDiffusion(
        x,
        q, 
        scale, 
        center, 
        radius, 
        DR
    )
    return np.convolve(model, resolution/resolution.sum(), mode='same')

# Fit
model_all_qs = []

for i in range(len(q_5A)):

    x = hw_5A[idx_5A]
    data = sqw_5A[idx_5A, i]
    error = err_5A[idx_5A, i]
    resol = res_5A[idx_5A, i]

    # Select only valid data (error = -1 for Q, w points not accessible)
    valid = np.where(error > 0.0)
    x = x[valid[1]]
    data = data[valid]
    error = error[valid]
    resol = resol[valid]

    # model
    model_q = bmp.Curve(
        model_convol, 
        x, data, error,
        name=f'q5A_{q_5A[i]:.2f}',
        q=q_5A[i], 
        scale=15, 
        center=0.0, 
        hwhm=0.1, 
        radius=1.1, 
        DR=1., 
        resolution=resol
    )

    # Fitted parameters
    model_q.scale.range(0, 1e2)
    model_q.center.range(-0.1, 0.1)
    model_q.hwhm.range(0., 1)
    model_q.radius.range(0.9, 1.1)
    model_q.DR.range(0.01, 5)

    # Q-independent parameters
    if i == 0:
        hwhm_q = model_q.hwhm
        R_q = model_q.radius
        DR_q = model_q.DR
    else:
        model_q.hwhm = hwhm_q
        model_q.radius = R_q
        model_q.DR = DR_q

    model_all_qs.append(model_q)

problem = bmp.FitProblem(model_all_qs)

### Display initial configuration: experimental data, fitting model with initial guesses

In [ ]:
slider = ipywidgets.IntSlider(value=0, min=0, max=len(q_5A)-1, continuous_update=False)
output = ipywidgets.Output()

def fig_q(model, ax, q_index=0):
    """
    Plot of experimental data, fitting model and residual for a selected q value
    
    Parameters
    ----------
    model: list of bumps.curve.Curves for all q
    
    ax: matplotlib.axes to be updated when changing the ipywidgets
    
    q_index: int
             index of q to be plotted
    
    """
    model = model[q_index]
    ax[0].errorbar(model.x,
                       model.y, 
                       yerr=model.dy,
                       label='experimental data',
                       color='C0')
    ax[0].plot(model.x,
                   model.theory(), 
                   label='theory (model)',
                   color='C1')
    ax[0].set_title(f'Model {model.name} - $\chi^2$={problem.chisq_str()}')
    ax[0].legend()
    ax[1].plot(model.x, model.residuals(), marker='o', linewidth=0, markersize=3, color='C0')
    

with output:
    fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True)
    ax[0].grid(); ax[1].grid()
    ax[1].set_ylabel('Residual')
    ax[1].set_xlabel(f"$\hbar \omega$")
    fig_q(model_all_qs, ax, 0)
    
    
def update_profile(change):
    """
    Update plots for a new q-value
    """
    with output:
        ax[0].clear(); ax[1].lines.clear()
        ax[0].grid()
        fig_q(model_all_qs, ax,change['new'])
                     
slider.observe(update_profile, names="value")

slider_label = ipywidgets.Label("q value to display")
slider_comp = ipywidgets.HBox([slider_label, slider])
ipywidgets.VBox([slider_comp, output])

In [ ]:
problem.summarize().splitlines()

### Choice of minimizer for bumps

In [ ]:
options_dict = {} 

for item in fitters.__dict__.keys():
    if item.endswith('Fit') and fitters.__dict__[item].id in fitters.FIT_AVAILABLE_IDS:
        options_dict[fitters.__dict__[item].name] = fitters.__dict__[item].id

w_choice_minimizer = ipywidgets.Dropdown(
    options=list(options_dict.keys()),
    value='Levenberg-Marquardt',
    description='Minimizer:',
    layout=ipywidgets.Layout(height='40px'))

w_choice_minimizer

### Number of steps for running fit using bumps 

In [ ]:
steps_fitting = ipywidgets.IntText(
    value=100,
    step=100,
    description='Number of steps when fitting',
    style={'description_width': 'initial'})

steps_fitting

## Running the fit

Run the fit using the *minimizer* defined above with a number of *steps* also specified above.

In [ ]:
# Preview of the settings
print('Initial chisq', problem.chisq_str())

In [ ]:
def settings_selected_optimizer(chosen_minimizer):
    """ 
    List the settings available for the selected optimizer
    
    This list can be used as arguments for the `fit` function
    """
    
    assert type(chosen_minimizer) == ipywidgets.widgets.widget_selection.Dropdown
    
    for item in fitters.__dict__.keys():
        if item.endswith('Fit') and \
        fitters.__dict__[item].id == options_dict[chosen_minimizer.value]:
            return [elt[0] for elt in fitters.__dict__[item].settings]

In [ ]:
print((f"With {w_choice_minimizer.value} optimizer, "
      f"you can use {settings_selected_optimizer(w_choice_minimizer)} as arguments of `fit`"))

In [ ]:
result = fitters.fit(
    problem,
    starts=10,
    keep_best=True,
    method=options_dict[w_choice_minimizer.value], 
    steps=int(steps_fitting.value)
)

## Showing the results

In [ ]:
problem.summarize().splitlines()

In [ ]:
# Other method to display the results of the fit (chi**2 and parameters' values)
print("final chisq", problem.chisq_str())
for k, v, dv in zip(problem.labels(), result.x, result.dx):
        print(k, ":", format_uncertainty_pm(v, dv))

### Display final configuration: experimental data, fitting model with output of fitting for the refined parameters

In [ ]:
slider1 = ipywidgets.IntSlider(value=0, min=0, max=len(q_5A)-1, continuous_update=False)
output1 = ipywidgets.Output()

with output1:
    fig1, ax1 = plt.subplots(nrows=2, ncols=1, sharex=True)
    ax1[0].grid(); ax1[1].grid()
    ax1[1].set_ylabel('Residual')
    ax1[1].set_xlabel(f"$\hbar \omega$")
    fig_q(model_all_qs, ax1, 0)
       
def update_profile1(change):
    """
    Update plots for a new q-value
    """
    with output1:
        ax1[0].clear(); ax1[1].lines.clear()
        ax1[0].grid()
        fig_q(model_all_qs, ax1, change['new'])
                     
slider1.observe(update_profile1, names="value")

slider1_label = ipywidgets.Label("q value to display")
slider1_comp = ipywidgets.HBox([slider1_label, slider1])
ipywidgets.VBox([slider1_comp, output1])